In [1]:
import json
import os
import tqdm

# hypers
dataset_names = ['inspec', 'krapivin', 'semeval', 'nus']
root_path = "/zf18/yw9fm/KPG_Project"
data_path = os.path.join(root_path,"data")
# DEC_MODEL = "decode_model_500000_1587513120"
DEC_MODEL = "decode_model_495000_1587512303"

In [7]:
import nltk
import spacy
stemmer = nltk.stem.porter.PorterStemmer()
spacy_nlp = spacy.load('en_core_web_sm')
doc = spacy_nlp(u'keyphrase generator')
for token in doc:
#     print(token.text)
    print(' '.join([stemmer.stem(w) for w in token.text.split()]))

keyphras
gener


In [2]:
# writing functions

# write test data and summary to a new .txt file
def write_aug_test(name, test_data):
    dec_dir = os.path.join(root_path, 'log', name, DEC_MODEL)
    gen_dir = os.path.join(dec_dir,'rouge_dec_dir')
    gen = os.listdir(gen_dir)
    gen = sorted(gen)
    target_txt_path = "../data/EN/"+name+".test.100aug.txt"
    with open(target_txt_path,'w+') as f_in:
        for idx, data in enumerate(test_data):
            read_path = os.path.join(gen_dir,gen[idx])
            assert os.path.exists(read_path)
            with open(read_path,'r') as f_out:
                tmp_sum = f_out.readlines()
            summ = ' $$ '.join([tmp.strip() for tmp in tmp_sum])
            abstract = data['abstract']
            title = data['title']
            art = '\n'.join((title, summ, abstract)).strip()
            f_in.write(art + '\n')

In [3]:
# write datasets to AutoPhrase framework
for name in dataset_names:
    test_path = os.path.join(data_path,name,name+'_test_spacynp.json')
    test_data = [json.loads(line) for line in open(test_path, 'r')]
    write_aug_test(name, test_data)
    print(name,os.path.exists(test_path),len(test_data))

inspec True 500
krapivin True 460
semeval True 100
nus True 211


In [8]:
import re
from tqdm import tqdm
import json
import math
# this part should be processed after autophrase.sh and phrasal_segmentation.sh

EVAL_K = [10, 50]

first_k = [100]

def prepare_aug(test_data, name,k ,model_name='kp20k'):
    # extract ranking score
#     print("extracting phrase quality scores")
    data_size = len(test_data)
    m_weight = {}
    s_weight = {}
    a_weight = {}
    multi_path = "../models/" + model_name +"/AutoPhrase_multi-words.txt"
    single_path = "../models/" + model_name +"/AutoPhrase_single-word.txt"
    auto_path = "../models/" + model_name +"/AutoPhrase.txt"
    with open(multi_path, 'r') as m:
        lines = [line.lower().rstrip().split('\t') for line in m]
    for score,key in lines:
        m_weight[key] = float(score)
    with open(single_path, 'r') as s:
        lines = [line.lower().rstrip().split('\t') for line in s]
    for score,key in lines:
        s_weight[key] = float(score)

    with open(auto_path, 'r') as s:
        lines = [line.lower().rstrip().split('\t') for line in s]
    for score,key in lines:
        a_weight[key] = float(score)
    
    # extract phrase and evaluate original text dataset
#     print("extracting phrases and calculate tf-idf")
    seg_path = "../models/" + model_name +"/segmentation."+name+".100aug.txt"
    
    with open(seg_path,'r') as f:
        lines = [line.lower().rstrip() for line in f]
    assert len(lines) == 3*data_size
    aug_tf = []
    absent_aug_tf = []
    phrase = set()
    ref = []
    for j in range(data_size):
        title_k = re.findall('<phrase>(.+?)</phrase>', lines[3*j])
        sum_k = ' '.join(lines[3*j+1].split('$$')[:k])
        sum_k = re.findall('<phrase>(.+?)</phrase>', sum_k)
        abstract_k = re.findall('<phrase>(.+?)</phrase>', lines[3*j+2])  
        counts = dict()
        count_absent = dict()
        for i in title_k:
            counts[i] = counts.get(i, 0) + 5
            phrase.add(i)
        for i in sum_k:
            counts[i] = counts.get(i, 0) + 0.5/100
            count_absent[i] = count_absent.get(i,0) + 1
            phrase.add(i)
        for i in abstract_k:
            counts[i] = counts.get(i, 0) + 1  
            phrase.add(i)
        # update tf with phrase quality
        for item in counts.keys():
            if item in m_weight:
                counts[item] = counts[item]*m_weight[item]
            elif item in s_weight:
                counts[item] = counts[item]*s_weight[item]*0.1
            else:
                counts[item] = counts[item]
        aug_tf.append(counts)
        absent_aug_tf.append(count_absent)
    if os.path.exists(os.path.join(model_name,name+'_aug_df'+str(k)+'.json')):
        aug_df = json.load(open(os.path.join(model_name,name+'_aug_df'+str(k)+'.json'),'r'))
    else:
        aug_df = {}
        for idx,kp in enumerate(phrase):
#             if idx % 1000 ==0:
#                 print(idx)
            for i in range(data_size):
                if kp in aug_tf[i]:
                    aug_df[kp] = aug_df.get(kp,0)+1
        with open(os.path.join(model_name,name+'_aug_df'+str(k)+'.json'), 'w') as fp:
            json.dump(aug_df, fp)
    assert len(phrase)<=len(aug_df)
        
    result = []
    for i in range(data_size):
        term_freq = aug_tf[i]
        score_dict = {}
        for key, value in term_freq.items():
            assert aug_df[key]>=1
            score_dict[key] = value * math.log(data_size/aug_df[key])
        result.append(sorted(score_dict, key=score_dict.get)[::-1])
        
    absent_result = []
    for i in range(data_size):
        term_freq = absent_aug_tf[i]
        score_dict = {}
        for key, value in term_freq.items():
            assert aug_df[key]>=1
            score_dict[key] = value * math.log(data_size/aug_df[key])
        absent_result.append(sorted(score_dict, key=score_dict.get)[::-1])
    
#     precision,recall = evaluate_oracle(present,result)
    print('Start Evaluating:')
    for score in EVAL_K:
        for gd in [absent]:
            print('top_k:',score)
            evaluate_recall(score,gd,absent_result)
#             evaluate(score,gd,result)
#     return result, precision, recall


def evaluate_recall(K, gold, result):
    precision = []
    recall = []
    assert len(gold)==len(result)
    for gt, rank in zip(gold, result):
        if len(gt)>=1:
            gt = [' '.join(ls) for ls in gt]
            ref = set()
            i = 0
            while len(ref)<K and len(rank)>i:
                ref.add(stem_process(rank[i]))
                i += 1
            tmp = 0
            for rk in ref:
                if rk in gt:
                    tmp += 1
            recall.append(tmp*1.0/len(gt))
            precision.append(tmp*1.0/K)
            
    p = sum(precision)/len(precision)
    r = sum(recall)/len(recall)
    print('recall:',r)
    if p > 0:
        print('f1:',2*p*r/(p+r))
def stem_process(text):
    doc = spacy_nlp(text)
    return ' '.join([stemmer.stem(w) for w in doc.text.split()])

In [9]:
# evaluation
for name in dataset_names:
    test_path = os.path.join(data_path,name,name+'_test_spacynp.json')
    test_data = [json.loads(line) for line in open(test_path, 'r')]
    # extract present/absent groundtruth
    present = [[[stemmer.stem(word) for word in words] for words in data['present_tgt_phrases']] for data in test_data]
    absent = [[[stemmer.stem(word) for word in words] for words in data['absent_tgt_phrases']] for data in test_data]
      
    
#     print(name,'evaluate normal:')
#     prepare_normal(test_data,name)
    print(name,'evlauate augmented data')
    for i in first_k:
        print(i)
        prepare_aug(test_data,name,i)

inspec evlauate augmented data
100
Start Evaluating:
top_k: 10
recall: 0.00020242914979757087
f1: 0.0002288329519450801
top_k: 50
recall: 0.018184885290148448
f1: 0.0012207597096449074
krapivin evlauate augmented data
100
Start Evaluating:
top_k: 10
recall: 0.001199040767386091
f1: 0.0006851661527920521
top_k: 50
recall: 0.028617106314948036
f1: 0.00247780396740865
semeval evlauate augmented data
100
Start Evaluating:
top_k: 10
recall: 0.0
top_k: 50
recall: 0.007295173961840628
f1: 0.0017745017745017745
nus evlauate augmented data
100
Start Evaluating:
top_k: 10
recall: 0.002307692307692308
f1: 0.0014201183431952664
top_k: 50
recall: 0.0282460506954434
f1: 0.0045284401727506305


doc.text.split()